# Data Cleaning Pipeline
This notebook outlines a structured process for cleaning, transforming, and preparing app usage data for analysis. 
The steps include removing unwanted text, converting time formats, reshaping data, and merging datasets.

---

## Step 1: Import Libraries

In [1]:
import pandas as pd
import re
import numpy as np

## Step 2: Load the Dataset
Load the primary dataset containing app usage data.

In [2]:
# File paths
file_path = "StayFree Export - Total Usage - 12_19_24.xls"  # Replace with your actual file path
output_path = "usage_data_cleaned.xlsx"

# Load the dataset
usage_time_df = pd.read_excel(file_path, sheet_name="Usage Time")
usage_time_df.head()

,Unnamed: 0,Device,"December 8, 2024","December 9, 2024","December 10, 2024","December 11, 2024","December 12, 2024","December 13, 2024","December 14, 2024","December 15, 2024","December 16, 2024","December 17, 2024","December 18, 2024","December 19, 2024",Total Usage
0,a.co,NaN,0s,0s,0s,0s,0s,0s,0s,0s,0s,0s,2s,0s,2s
1,ActionDash,NaN,0s,0s,0s,0s,0s,0s,0s,0s,0s,0s,1m 47s,44s,2m 32s
2,Airtel,NaN,0s,0s,0s,0s,0s,12s,0s,0s,0s,0s,0s,0s,12s
3,amazon.ca,NaN,0s,0s,0s,0s,0s,0s,0s,0s,0s,0s,2s,0s,2s
4,analytics.twitter.com,NaN,0s,0s,0s,0s,0s,0s,0s,0s,0s,0s,6s,0s,6s


## Step 3: Remove Dynamic Text Information
Define a function to clean dynamic text, such as creation dates and app metadata.

In [3]:
# Function to remove dynamic creation info
def remove_creation_info(text):
    if isinstance(text, str):
        text = re.sub(r"Creation date:\s*\d{1,2}/\d{1,2}/\d{2,4}\s*\d{1,2}:\d{2}:\d{2}(:\d{2})?", '', text)
        text = re.sub(r"Created by \u201cStayFree\u201d\.", '', text)
    return text

# Apply the function to clean the dataset
usage_time_df = usage_time_df.applymap(remove_creation_info)
usage_time_df.columns.values[0] = 'Apps'

C:\Users\Lakshay\AppData\Local\Temp\ipykernel_23312\3641865516.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  usage_time_df = usage_time_df.applymap(remove_creation_info)


## Step 4: Clean Unwanted Columns
Remove columns with irrelevant data (e.g., `Unnamed` or `Device`) to streamline the dataset.

In [4]:
# Remove unwanted columns
usage_time_df = usage_time_df.loc[:, ~usage_time_df.columns.str.contains('^Unnamed')]
usage_time_df = usage_time_df.loc[:, ~usage_time_df.columns.str.contains('Device')]

## Step 5: Convert Time to Hours
Define a function to standardize various time formats (e.g., `3h 4m`, `29m`) into total hours.

In [5]:
# Function to convert time strings to hours
def convert_to_hours(time_str):
    if pd.isna(time_str) or time_str == '0s':
        return 0.0

    hours = minutes = seconds = 0
    
    match = re.match(r'(\d+)h\s*(\d+)m\s*(\d+)s', time_str)
    if match:
        hours, minutes, seconds = int(match.group(1)), int(match.group(2)), int(match.group(3))
    else:
        match = re.match(r'(\d+)h\s*(\d+)m', time_str)
        if match:
            hours, minutes = int(match.group(1)), int(match.group(2))
        else:
            match = re.match(r'(\d+)m\s*(\d+)s', time_str)
            if match:
                minutes, seconds = int(match.group(1)), int(match.group(2))
            else:
                match = re.match(r'(\d+)m', time_str)
                if match:
                    minutes = int(match.group(1))
                else:
                    match = re.match(r'(\d+)s', time_str)
                    if match:
                        seconds = int(match.group(1))

    return hours + (minutes / 60) + (seconds / 3600)

# Test the function
convert_to_hours("29m")

0.48333333333333334

## Step 6: Clean DataFrame
Remove rows with `Total Usage`, empty values, or NaN entries.

In [6]:
# Clean the data
usage_time_cleaned = usage_time_df.drop('Total Usage', axis=1, errors='ignore')
usage_time_cleaned = usage_time_cleaned.dropna(how='any')
usage_time_cleaned = usage_time_cleaned.loc[~(usage_time_cleaned == '').any(axis=1)]
usage_time_cleaned = usage_time_cleaned[usage_time_cleaned['Apps'] != 'Total Usage']

## Step 7: Reshape DataFrame to Long Format
Transform the data from a wide format (dates as columns) to a long format.

In [7]:
# Convert to long format
usage_time_long = pd.melt(usage_time_cleaned, id_vars=['Apps'], var_name='Date', value_name='Usage')

## Step 8: Convert Date Column
Standardize the `Date` column to ISO format (`YYYY-MM-DD`).

In [8]:
# Format the Date column
usage_time_long['Date'] = pd.to_datetime(usage_time_long['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

## Step 9: Convert Usage Column to Hours
Apply the `convert_to_hours` function to the `Usage` column.

In [9]:
# Convert usage to hours
usage_time_long['Usage (in hours)'] = usage_time_long['Usage'].apply(convert_to_hours)

## Step 10: Add Usage Count Data
Load usage count data and merge it with the cleaned dataset.

In [10]:
# Load and clean usage count data
usage_count_df = pd.read_excel(file_path, sheet_name="Usage Count")
usage_count_df = usage_count_df.applymap(remove_creation_info)
usage_count_df = usage_count_df.loc[~(usage_count_df == '').any(axis=1)]
usage_count_df = usage_count_df.drop(columns=['Device'], errors='ignore')
usage_count_df = usage_count_df.dropna()
usage_count_df.rename(columns={usage_count_df.columns[0]: 'App Name'}, inplace=True)
usage_count_df = usage_count_df[usage_count_df['App Name'] != 'Total Usage']

# Reshape to long format
usage_count_long = usage_count_df.melt(id_vars=['App Name'], var_name='Date', value_name='Usage Count')
usage_count_long['Date'] = pd.to_datetime(usage_count_long['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Merge with the main dataset
merged_df = pd.merge(usage_time_long, usage_count_long, left_on=['Apps', 'Date'], right_on=['App Name', 'Date'], how='left')
merged_df = merged_df.drop(columns=['App Name']).fillna({'Usage Count': 0})

C:\Users\Lakshay\AppData\Local\Temp\ipykernel_23312\4268449071.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  usage_count_df = usage_count_df.applymap(remove_creation_info)


## Step 11: Add Device Unlock Data
Load and merge device unlock data into the main dataset.

In [18]:
# Load and clean unlock data
unlock_df = pd.read_excel(file_path, sheet_name="Device Unlocks")
unlock_df = unlock_df.applymap(remove_creation_info)
unlock_df = unlock_df.dropna()
unlock_df = unlock_df.drop(columns=['Unnamed: 0', 'Total Usage'], errors='ignore')
# Reshape unlock data
unlock_long = unlock_df.melt(var_name="Date", value_name="Device Unlock")
unlock_long["Date"] = pd.to_datetime(unlock_long["Date"], format="%B %d, %Y").dt.strftime("%Y-%m-%d")

# Merge unlock data
merged_df = pd.merge(merged_df, unlock_long, on="Date", how="left")
merged_df["Device Unlock"] = merged_df["Device Unlock"].fillna(0)

C:\Users\Lakshay\AppData\Local\Temp\ipykernel_23312\1689293242.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  unlock_df = unlock_df.applymap(remove_creation_info)


## Step 12: Save the Cleaned Data
Save the final cleaned dataset to an Excel file.

In [19]:
# Save the final DataFrame
merged_df.to_excel(output_path, index=False)
merged_df.head()

,Apps,Date,Usage,Usage (in hours),Usage Count,Device Unlock
0,a.co,2024-12-08,0s,0.0,0.0,266.0
1,ActionDash,2024-12-08,0s,0.0,0.0,266.0
2,Airtel,2024-12-08,0s,0.0,0.0,266.0
3,amazon.ca,2024-12-08,0s,0.0,0.0,266.0
4,analytics.twitter.com,2024-12-08,0s,0.0,0.0,266.0
